

## 0. 라이브러리 설치



In [ ]:
!pip install pycaret==2.3.10

     |████████████████████████████████| 320 kB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 46.0 MB/s 
     |████████████████████████████████| 262 kB 56.1 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 118 kB 59.1 MB/s 
     |████████████████████████████████| 1.7 MB 42.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.8 MB 35.8 MB/s 
     |████████████████████████████████| 167 kB 48.3 MB/s 
     |████████████████████████████████| 16.8 MB 37.6 MB/s 
     |████████████████████████████████| 88 kB 6.9 MB/s 
     |████████████████████████████████| 2.0 MB 41.5 MB/s 
     |████████████████████████████████| 636 kB 74.4 MB/s 
     |████████████████████████████████| 102 kB 12.6 MB/s 
     |████████████████████████████████| 690 kB 70.9 MB/s 
     |█████████████████

In [ ]:
!pip install jinja2==3.1.2

     |████████████████████████████████| 133 kB 5.1 MB/s 
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install xgboost==1.6.0

     |████████████████████████████████| 193.7 MB 30 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


## 1. 데이터 불러오기
전처리된 데이터 csv파일로 저장해 구글 드라이브에 올려놓고 가져와 사용하는 방식
- index, occyp_type, FLAG_MOBIL 삭제
- gender, car, reality -> 0과 1로 변경
- DAYS_BIRTH -> age 컬럼 생성 후 삭제
- DAYS_EMPLOYED -> (-1)곱하고, 음수는 0으로 변경
- begin_month -> (-1) 곱해줌
- income_total -> 로그스케일 적용

In [ ]:
from google.colab import drive

drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
import io
import pandas as pd

In [ ]:
file = '/gdrive/MyDrive/Colab Notebooks/Semi_project_03/df_occyp예측.csv'
df = pd.read_csv(file)
df.drop('DAYS_BIRTH', axis = 1, inplace = True)
df.head(3)

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,age
0,0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,4709,1,0,0,0,Laborers,2.0,6.0,1.0,38
1,1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,1,0,0,1,Laborers,3.0,5.0,1.0,31
2,2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,4434,1,0,1,0,Managers,2.0,22.0,2.0,52


In [ ]:
df

,index,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit,age
0,0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,4709,1,0,0,0,Laborers,2.0,6.0,1.0,38
1,1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,1,0,0,1,Laborers,3.0,5.0,1.0,31
2,2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,4434,1,0,1,0,Managers,2.0,22.0,2.0,52
3,3,0,0,1,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,2092,1,0,1,0,Sales staff,2.0,37.0,0.0,41
4,4,0,1,1,0,157500.0,State servant,Higher education,Married,House / apartment,2105,1,0,0,0,Managers,2.0,26.0,2.0,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,26452,0,0,0,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,1984,1,0,0,0,Core staff,4.0,2.0,1.0,33
26453,26453,0,0,1,1,180000.0,Working,Higher education,Separated,House / apartment,2475,1,0,0,0,Managers,2.0,47.0,2.0,41
26454,26454,0,1,0,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,2015,1,0,0,0,Core staff,2.0,25.0,2.0,27
26455,26455,1,0,1,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,107,1,0,0,0,Laborers,1.0,59.0,2.0,27


## 2. pycaret 적용

In [ ]:
from pycaret.classification import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
model = setup(data= df,
              target='credit',
              numeric_features=['DAYS_EMPLOYED','child_num'], # categoric으로 분류돼서 직접 지정
              train_size=0.7,
              session_id=42)

,Description,Value
0,session_id,42
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 20)"
5,Missing Values,False
6,Numeric Features,7
7,Categorical Features,12
8,Ordinal Features,False
9,High Cardinality Features,False


## 2-1. 분류 모델 비교

In [ ]:
# 평가 지표 추가
from sklearn.metrics import log_loss

add_metric('logloss','LogLoss',log_loss,greater_is_better=False,target='pred_proba')

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7f85412ffd40>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [ ]:
# svm과 ridge 모델은 predict_proba를 지원하지 않아 제외
top_3_models = compare_models(sort='logloss', n_select=3, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7031,0.7112,0.4537,0.6846,0.6443,0.2750,0.3327,0.7613,10.229
lightgbm,Light Gradient Boosting Machine,0.7000,0.7040,0.4270,0.6959,0.6245,0.2391,0.3218,0.7647,1.018
gbc,Gradient Boosting Classifier,0.6924,0.6543,0.4086,0.6387,0.6074,0.2075,0.2962,0.7939,6.924
lda,Linear Discriminant Analysis,0.6451,0.6108,0.3395,0.5362,0.5156,0.0178,0.0588,0.8590,0.208
nb,Naive Bayes,0.6494,0.6074,0.3423,0.5777,0.5192,0.0276,0.0987,0.8643,0.049
lr,Logistic Regression,0.6435,0.5990,0.3333,0.4142,0.5040,-0.0001,-0.0014,0.8669,0.818
dummy,Dummy Classifier,0.6436,0.5000,0.3333,0.4142,0.5040,0.0000,0.0000,0.8792,0.035
rf,Random Forest Classifier,0.6988,0.7423,0.5323,0.6760,0.6806,0.3611,0.3686,0.9672,2.505
ada,Ada Boost Classifier,0.6927,0.6254,0.4049,0.6221,0.6038,0.2001,0.3005,1.0808,0.690
et,Extra Trees Classifier,0.6704,0.7040,0.5137,0.6486,0.6550,0.3106,0.3153,2.4374,2.669


In [ ]:
# xgboost모델 예측 결과값
predict_model(top_3_models[0])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Extreme Gradient Boosting,0.699,0.7203,0.4515,0.6765,0.6373,0.2764,0.3363,0.7607


,child_num,income_total,DAYS_EMPLOYED,family_size,begin_month,age,gender_0,car_0,reality_0,income_type_Commercial associate,...,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,credit,Label,Score
0,0.0,315000.0,545.0,1.0,11.0,28.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.7128
1,1.0,216000.0,1782.0,2.0,7.0,28.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.6496
2,0.0,238500.0,7095.0,2.0,53.0,41.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.8197
3,0.0,112500.0,0.0,2.0,14.0,66.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.6676
4,1.0,247500.0,1931.0,3.0,16.0,35.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.6522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7933,0.0,135000.0,0.0,1.0,-0.0,63.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1,0.5873
7934,0.0,270000.0,1674.0,2.0,29.0,26.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.6101
7935,0.0,135000.0,1259.0,2.0,24.0,28.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.5524
7936,0.0,117000.0,0.0,2.0,39.0,63.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.7019


## 2-2. 분류 모델 튜닝

In [ ]:
tuned_top3 = [tune_model(i, optimize='logloss') for i in top_3_models]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.7079,0.7073,0.4850,0.6860,0.6645,0.3164,0.3534,0.7761
1,0.7019,0.7033,0.4797,0.6730,0.6595,0.3084,0.3402,0.7821
2,0.7003,0.7178,0.4796,0.6724,0.6581,0.3046,0.3368,0.7786
3,0.6911,0.6888,0.4610,0.6528,0.6451,0.2797,0.3119,0.8050
4,0.7052,0.7259,0.4739,0.6798,0.6581,0.3024,0.3450,0.7714
5,0.7009,0.7312,0.4777,0.6703,0.6579,0.3063,0.3377,0.7592
6,0.7095,0.7394,0.4870,0.6854,0.6668,0.3215,0.3588,0.7536
7,0.6874,0.7124,0.4526,0.6487,0.6374,0.2627,0.2971,0.7856
8,0.6971,0.7223,0.4639,0.6600,0.6492,0.2895,0.3247,0.7654


In [ ]:
top_3_models

[XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
               missing=nan, monotone_constraints='()', n_estimators=100,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=0, ...),
 LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
                importance_type='split', learning_rate=0.1, max_depth=-1,
                min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
                n_estimators=100, n_job

In [ ]:
tuned_top3

[XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=2,
               missing=nan, monotone_constraints='()', n_estimators=170,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=10, ...),
 LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=6, min_child_wei

## 2-3. 분류 모델 블렌딩

In [ ]:
blend_top_3 = blend_models(estimator_list=tuned_top3, fold=5, optimize='logloss')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.7122,0.7068,0.4592,0.7083,0.6519,0.2924,0.3589,0.7531
1,0.6984,0.7028,0.4377,0.6734,0.6325,0.2548,0.3161,0.7668
2,0.7084,0.7259,0.4489,0.7059,0.6434,0.2769,0.3479,0.7519
3,0.7071,0.7184,0.4500,0.6891,0.6444,0.2801,0.3435,0.7533
4,0.7062,0.7154,0.4489,0.6959,0.6430,0.2775,0.3395,0.7523
Mean,0.7065,0.7139,0.4489,0.6945,0.6430,0.2763,0.3412,0.7555
Std,0.0045,0.0083,0.0068,0.0126,0.0062,0.0122,0.0141,0.0057


In [ ]:
# 블렌딩 모델로 예측
pred = predict_model(blend_top_3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.7027,0.7329,0.4526,0.6899,0.6394,0.2811,0.3463,0.7479


## 2.4 마무리

In [ ]:
# finalize_model : 전체 데이터로 마지막 학습을 진행함
final_blended = finalize_model(blend_top_3)

In [ ]:
final_blended

VotingClassifier(estimators=[('xgboost',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=0.7,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=0,
                                            gpu_id=-1, grow_policy='depthwise',
                                            importance_type=None,
                                            interaction_constraints='',
                                            learning_rate=0.3, max_bin=256,...
                                                         min_impurity_decrease=0.001,
                                                         min_impurity_spli

In [ ]:
# 최종 모델로 예측
predict_model(final_blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.7502,0.9058,0.5283,0.7773,0.706,0.4101,0.48,0.5842


,child_num,income_total,DAYS_EMPLOYED,family_size,begin_month,age,gender_0,car_0,reality_0,income_type_Commercial associate,...,occyp_type_Medicine staff,occyp_type_Private service staff,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_Waiters/barmen staff,credit,Label,Score
0,0.0,315000.0,545.0,1.0,11.0,28.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.6838
1,1.0,216000.0,1782.0,2.0,7.0,28.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.5877
2,0.0,238500.0,7095.0,2.0,53.0,41.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.7410
3,0.0,112500.0,0.0,2.0,14.0,66.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.5799
4,1.0,247500.0,1931.0,3.0,16.0,35.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.6805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7933,0.0,135000.0,0.0,1.0,-0.0,63.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0,0.4975
7934,0.0,270000.0,1674.0,2.0,29.0,26.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.6929
7935,0.0,135000.0,1259.0,2.0,24.0,28.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2,0.5766
7936,0.0,117000.0,0.0,2.0,39.0,63.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2,0.7945


## 2.5 모델 저장

In [ ]:
save_model(final_blended, '/gdrive/MyDrive/Colab Notebooks/Semi_project_03/final_blended_model_occupy_SMOTE x')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=['index'],
                                       ml_usecase='classification',
                                       numerical_features=['DAYS_EMPLOYED',
                                                           'child_num'],
                                       target='credit', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_...
                                                                           min_impurity_decrease=0.001,
                                                                           min_impurity_split=None,
                                            